<a href="https://colab.research.google.com/github/IrisMeng9/2024_thematic_investing/blob/main/Stock_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook details the complete process of stock selection in thematic investing, which is divided into three parts. We examine two themes: AI and Renewable Energy. The first two steps are the same for both themes, while the third step of selections is different.

1. We tested 4 different search methods with Elasticsearch to find companies **generally related** to a specific theme. In this step, we displayed the first 200 search results for each method. The dataset is named as 'reference company desc'. You can skip the search process.   
2. After having the search resutls from 4 different methods, we
manually reviewed the results, labeling the companies as 1 if they are theme-related and 0 otherwise. This manual review was still relatively general, it allowed us to compare the accuracy of the four methods and select the best two: **ELSER and Exact Search**. We combined the data from these two sets of results to form a union set, then filtered out the data that was labeled as 1 in either set. These companies were used as the dataset for the next step.   
3. As mentioned before, the methods used for sorting AI and Renwable Energy stocks were different.

    3.1  For AI:    
    In this step, we split the description of each company into individual sentences, use minilm model to transform the sentence and identify those related to specific theme. Then, we used BART zero-shot classification to label these sentences. We need to define our own labels, each associated with a score. After classfication, we add up the scores for each company and rank them in descending order to create our recommend list.   
    
    3.2  For Renewable Energy:
    
    In this step, we split the description of each company into sentences, and tag description based on theme keywords. Setting score rubric and scoring each of the company depends on their tags. Remove all the comapny with low irrelevance. After classfication, we add up the scores for each company and rank them in descending order to create our recommend list.  



## Step 1: 4 search methods
In this section, we present 4 search methods. Exact and BM25 are fast, while vector search and Elser will take longer. You can skip this cause we already have search results saved to a file.


In [ ]:
pip install elasticsearch

### Exact match

In [ ]:
import elasticsearch as es
from elasticsearch.helpers import BulkIndexError, bulk
import time
import pandas as pd
import numpy as np

# Elasticsearch
es_idx_utestiam = 'utest-companydes'
cnxn_es = es.Elasticsearch(
    cloud_id='adt-search-es-dev:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDg1NjQzOWRkZjE1NjQzYTE5NzBlYjEwYTg1NDYyMTY4JGU2OWRmMmQ4MGJlOTRkMDc5ZGZkY2ZmZDNhMGUwYTBj',
    api_key='S3ZuM21JOEJabFl1cG5MZEE0Tm46ejMwb2NtcW5TWi1xWW1OU1ZCRi0wQQ=='
)

start_time = time.time()  # timer start

client_info = cnxn_es.info()
print('Connected to Elasticsearch!')
print(client_info.body)

index_mapping = {
    "properties": {
        "symbol": {"type": "keyword"},
        "marketcap": {"type": "long"},
        "cik": {"type": "integer"},
        "description": {"type": "text"},
        "sector": {"type": "text"},
        "industry": {"type": "text"},
        "country": {"type": "keyword"},
    }
}

# delete old and create new index
cnxn_es.indices.delete(index=es_idx_utestiam, ignore_unavailable=True)
cnxn_es.indices.create(index=es_idx_utestiam, mappings=index_mapping)
print(f"Created index {es_idx_utestiam}.")

# read/load to Elasticsearch
file_path = '/content/drive/MyDrive/thematic/reference company desc.xlsx'
dfg = pd.read_excel(file_path).replace({np.nan: None, pd.NA: None, " ": None})
dfg = dfg.fillna('')

actions = []
for index, row in dfg.iterrows():
    d = {
        "_index": es_idx_utestiam,
        "_op_type": 'index',
        "_source": {
            "symbol": row["Symbol"],
            "marketcap": row["MktCap"],
            "cik": row["cik"],
            "description": row["description"],
            "sector": row["sector"],
            "industry": row["industry"],
            "country": row["country"],
        }
    }
    actions.append(d)
try:
    bulk(cnxn_es, actions)
    print("Data loaded successfully.")
except BulkIndexError as e:
    print(f"Bulk indexing error: {e}")
    for error in e.errors:
        print(error)
except Exception as e:
    print(f"Error loading data: {e}")

cnxn_es.indices.refresh(index=es_idx_utestiam)
result = cnxn_es.cat.count(index=es_idx_utestiam, format="json")
print("Document count:", result)

# def 200 companies
def get_search_results(query, index, size=200, scroll='2m'):
    data = []
    result = cnxn_es.search(index=index, body=query, size=100, scroll=scroll)
    scroll_id = result['_scroll_id']
    hits = result['hits']['hits']
    while len(hits) > 0 and len(data) < size:
        for doc in hits:
            if len(data) >= size:
                break
            record = {
                "Document ID": doc['_id'],
                "Score": doc['_score']
            }
            record.update(doc['_source'])
            data.append(record)
        result = cnxn_es.scroll(scroll_id=scroll_id, scroll=scroll)
        scroll_id = result['_scroll_id']
        hits = result['hits']['hits']
    return data

# Boolean query
query_re = {
    "query": {
        "bool": {
            "should": [
                {"match": {"description": "renewable"}},
                {"match": {"description": "solar"}},
                {"match": {"description": "wind"}},
                {"match": {"description": "hydro"}},
                {"match": {"description": "biomass"}},
                {"match": {"description": "geothermal"}},
                {"match": {"description": "tidal"}},
                {"match": {"description": "photovoltaic"}},
                {"match": {"description": "sustainable energy"}},
                {"match": {"description": "green energy"}}
            ],
            "minimum_should_match": 1
        }
    },
    "track_scores": True
}

query_ai = {
    "query": {
        "bool": {
            "should": [
                {"match": {"description": "machine learning"}},
                {"match": {"description": "cloud"}},
                {"match": {"description": "financial technology"}},
                {"match": {"description": "natural language processing"}},
                {"match": {"description": "computer vision"}},
                {"match": {"description": "deep learning"}},
                {"match": {"description": "autonomous vehicles"}},
                {"match": {"description": "AI hardware"}},
                {"match": {"description": "reinforcement learning"}},
                {"match": {"description": "data mining"}},
                {"match": {"description": "Chatbots"}},
                {"match": {"description": "intelligent systems"}},
                {"match": {"description": "robotics"}},
                {"match": {"description": "AI algorithms"}},
                {"match": {"description": "AI research"}},
                {"match": {"description": "AI applications"}}
            ],
            "minimum_should_match": 1
        }
    },
    "track_scores": True
}



In [ ]:
# AI
start_ai_time = time.time()
data_ai = get_search_results(query_ai, es_idx_utestiam)
end_ai_time = time.time()

df_ai = pd.DataFrame(data_ai)
output_file_ai = 'AI_SearchResults.xlsx'
df_ai.to_excel(output_file_ai, index=False, sheet_name='Exact Match')
print(f"Search results saved to {output_file_ai}")
print(f"AI search time: {end_ai_time - start_ai_time:.2f} seconds")

In [ ]:
# Renewable Energy
start_re_time = time.time()
data_re = get_search_results(query_re, es_idx_utestiam)
end_re_time = time.time()

df_re = pd.DataFrame(data_re)
output_file_re = 'Renewable_SearchResults.xlsx'
df_re.to_excel(output_file_re, index=False, sheet_name='Exact Match')
print(f"Search results saved to {output_file_re}")
print(f"Renewable Energy search time: {end_re_time - start_re_time:.2f} seconds")

### BM25

In [ ]:
##BM25
def get_index_count(index_name):
    return cnxn_es.cat.count(index=index_name, format="json")

def search_es(query_string, index_name):
    query_body = {
        "size":200,
        "query": {
            "multi_match": {
                "query": query_string,
                "fields": ["description"],  # Specify fields to search
                "fuzziness": "AUTO",
            }
        }
    }
    response = cnxn_es.search(index=index_name, body=query_body)
    return response

# Check index count
index_count = get_index_count(es_idx_utestiam)
print("Number of documents in index:", index_count)

# Perform search
theme_AI = "AI, Artificial Intelligence, machine learning, neural networks, deep learning, cloud computing, robotics"

search_result = search_es(theme_AI, es_idx_utestiam)
print("Search Result:", search_result)

data = []
for doc in search_result['hits']['hits']:
    source = doc['_source']
    source['document_id'] = doc['_id']  # Add document ID to the source data
    source['score'] = doc['_score']  # Add score to the source data
    data.append(source)

# Create DataFrame
df = pd.DataFrame(data)

output_file = 'AI_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='BM25', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

In [ ]:
theme_renew = "renewable energy, clean energy, solar energy, wind energy, hydroelectric power, biomass energy, geothermal energy, clean energy, sustainable energy"

search_result = search_es(theme_renew, es_idx_utestiam)
print("Search Result:", search_result)

data = []
for doc in search_result['hits']['hits']:
    source = doc['_source']
    source['document_id'] = doc['_id']  # Add document ID to the source data
    source['score'] = doc['_score']  # Add score to the source data
    data.append(source)

# Create DataFrame
df = pd.DataFrame(data)

output_file = 'Renewable_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='BM25', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

### Vector search

In [ ]:
# pip install sentence-transformers
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
# ELASTIC_CLOUD_ID = cloud_id
# ELASTIC_API_KEY = api_key


index_mapping = {
    "properties": {
        "symbol": {"type": "keyword"},
        "marketcap": {"type": "long"},
        "cik": {"type": "integer"},
        "description": {"type": "text"},
        "sector": {"type": "text"},
        "industry": {"type": "text"},
        "country": {"type": "keyword"},
        "embedding": {'type': 'dense_vector',
                      "dims": 384,
                      "index": True,
                      "similarity": "l2_norm",}, # for vector embedding
        # "elser_embedding": {'type': 'sparse_vector'}, # for ELSER model
    }
}


# create pipeline
cnxn_es.ingest.put_pipeline(
    id="vector-pipeline",
    description="Ingest pipeline for vector search",
    processors=[
        {
            "inference": {
                "model_id": "sentence-transformers__all-minilm-l6-v2",
                "input_output": [
                    {"input_field": "description", "output_field": "embedding"}
                ],
            }
        }
    ],
)

print("Pipeline vector-pipeline has been created successfully.")

index_setting = {
    "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
        "default_pipeline": "vector-pipeline",
    }
}


SHOULD_DELETE_INDEX = True
if SHOULD_DELETE_INDEX:
    if cnxn_es.indices.exists(index=es_idx_utestiam):
        print("Deleting existing %s" % es_idx_utestiam)
        cnxn_es.indices.delete(index=es_idx_utestiam, ignore_unavailable=True)

print("Creating index %s" % es_idx_utestiam)
cnxn_es.indices.create(
    index=es_idx_utestiam, mappings=index_mapping, settings=index_setting
)



In [ ]:
dfg = pd.read_excel("reference company desc.xlsx").replace({np.nan: None, pd.NA: None, " ": None})
dfg = dfg.fillna('')

# # Replace NaN values with None to ensure compatibility with JSON, as JSON uses 'null' to represent missing values.
actions = []
for index, row in dfg.iterrows():
    d = {
        "_index": es_idx_utestiam,
        "_op_type": 'index',
        "_source": {
            "symbol": row["Symbol"],
            "marketcap": row["MktCap"],
            "cik": row["cik"],
            "description": row["description"],
            "sector": row["sector"],
            "industry": row["industry"],
            "country": row["country"],
        }
    }
    actions.append(d)


# for test, large data size will have time our error
try:
    eshelpers.bulk(cnxn_es, actions, pipeline="vector-pipeline")
    print("Data loaded successfully.")
except BulkIndexError as e:
    print(f"Bulk indexing error: {e}")
    for error in e.errors:
        print(error)
except Exception as e:
    print(f"Error loading data: {e}")


cnxn_es.indices.refresh(index=es_idx_utestiam)
result = cnxn_es.cat.count(index=es_idx_utestiam, format="json")
print("Count Result:", result)


In [ ]:
def create_cosine_similarity_query(query_vector):
    return {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

def esquery(query_vector, idx=None):
    idx = idx if idx else es_idx_transcripts
    query = create_cosine_similarity_query(query_vector)

    r = cnxn_es.search(index=idx, body={
        "_source": {
            "excludes": []
        },
        "query": query,
        "size": 200
    })
    print(f"Total hits: {r['hits']['total']['value']}")  # Adjusted for Elasticsearch 7.x

    for hit in r['hits']['hits']:
        source = hit['_source']
        score = hit['_score']  # Accessing the score for each document
        print(f"Symbol: {source.get('symbol', 'N/A')}, Market Cap: {source.get('marketcap', 'N/A')}, \
    CIK: {source.get('cik', 'N/A')}, Description: {source.get('description', 'N/A')}, Score: {score}")

    # Normalize the results into a DataFrame
    df = pd.json_normalize(r['hits']['hits'])

    rows = []

    # Iterate over each hit and collect required details
    for hit in r['hits']['hits']:
        source = hit['_source']
        row = {
            "score": hit['_score'],
            "symbol": source.get('symbol', 'N/A'),
            "marketcap": source.get('marketcap', 'N/A'),
            "cik": source.get('cik', 'N/A'),
            "description": source.get('description', 'N/A'),
            "sector": source.get('sector', 'N/A'),
            "industry": source.get('industry', 'N/A'),
            "country": source.get('country', 'N/A')
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    # df.to_excel('vector_search_results.xlsx', index=False)

    return r, df

In [ ]:
ai_query_text = "AI, Artificial Intelligence, machine learning, neural networks, deep learning"

model = SentenceTransformer('all-MiniLM-L6-v2')
query_vector = model.encode(ai_query_text)

# Call the esquery function with the query vector
response, df = esquery(query_vector, es_idx_utestiam)

output_file = 'AI_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Vector', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

In [ ]:
renew_query_text = ("renewable energy, solar energy, wind energy, hydroelectric power, biomass energy, geothermal energy, "
               "clean energy, sustainable energy")

query_vector = model.encode(ai_query_text)

# Call the esquery function with the query vector
response, df = esquery(query_vector, es_idx_utestiam)

output_file = 'Renewable_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Vector', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

### ELSER model (from elastic search)
Notice: You will need about 30 mins to finish the search with ELSER model, the following codes are for demenstration, you can skip it

In [ ]:
from elasticsearch import exceptions
import time

index_mapping = {
    "properties": {
        "symbol": {"type": "keyword"},
        "marketcap": {"type": "long"},
        "cik": {"type": "integer"},
        "description": {"type": "text"},
        "sector": {"type": "text"},
        "industry": {"type": "text"},
        "country": {"type": "keyword"},
        'elser_embedding': {'type': 'sparse_vector'}, # for ELSER model
    }
}


try:
    cnxn_es.ml.delete_trained_model(model_id=".elser_model_2", force=True)
    print("Model deleted successfully. Proceeding with model creation.")
except exceptions.NotFoundError:
    print("Model does not exist. Proceeding with model creation.")

# create model
cnxn_es.ml.put_trained_model(
    model_id=".elser_model_2",
    input={"field_names": ["text_field"]}
)

# define model
while True:
    status = cnxn_es.ml.get_trained_models(model_id=".elser_model_2", include="definition_status")
    if status["trained_model_configs"][0]["fully_defined"]:
        print("ELSER Model is downloaded and ready to be deployed.")
        break
    else:
        print("Waiting for ELSER Model to be fully defined.")
    time.sleep(5)

# deploy
cnxn_es.ml.start_trained_model_deployment(
    model_id=".elser_model_2",
    number_of_allocations=1,
    wait_for="starting"
)

# deployment status
while True:
    status = cnxn_es.ml.get_trained_models_stats(model_id=".elser_model_2")
    if status["trained_model_stats"][0]["deployment_stats"]["state"] == "started":
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model deployment is in progress.")
    time.sleep(5)

# create pipeline
cnxn_es.ingest.put_pipeline(
    id="elser-ingest-pipeline",
    description="Ingest pipeline for ELSER",
    processors=[
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {"input_field": "description", "output_field": "elser_embedding"}
                ],
            }
        }
    ],
)

print("Pipeline 'elser-ingest-pipeline' has been created successfully.")

# delete the index first in case it already exists
cnxn_es.indices.delete(index=es_idx_utestiam, ignore_unavailable=True)
cnxn_es.indices.create(index=es_idx_utestiam,
                           settings={"index": {"default_pipeline": "elser-ingest-pipeline"}},
                           mappings=index_mapping,
                           )

In [ ]:
import pandas as pd
import numpy as np
import elasticsearch.helpers as eshelpers
from elasticsearch.helpers import BulkIndexError


dfg = pd.read_excel("reference company desc.xlsx").replace({np.nan: None, pd.NA: None, " ":None})
# Specifically fill missing values in the 'description' column with an empty string
dfg = dfg.fillna('')

# dfg = dfg[:500]


actions = []
for index, row in dfg.iterrows():
    d = {
        "_index": es_idx_utestiam,
        "_op_type": 'index',
        "_source": {
            "symbol": row["Symbol"],
            "marketcap": row["MktCap"],
            "cik": row["cik"],
            "description": row["description"],
            "sector": row["sector"],
            "industry": row["industry"],
            "country": row["country"],
        }
    }
    actions.append(d)


batch_size = 100

def load_batch(es_connection, actions):
    try:
        eshelpers.bulk(es_connection, actions)
        print("Batch loaded successfully.")
    except BulkIndexError as e:
        print(f"Bulk indexing error: {e}")
        for error in e.errors:
            print(error)
    except Exception as e:
        print(f"Error loading data: {e}")

# smaller size
for i in range(0, len(actions), batch_size):
    batch = actions[i:i + batch_size]
    load_batch(cnxn_es, batch)

cnxn_es.indices.refresh(index=es_idx_utestiam)
result = cnxn_es.cat.count(index=es_idx_utestiam, format="json")
print("Count Result:", result)

In [ ]:
def esquery1(query, idx=None):
    idx = idx if idx else es_idx_transcripts

    r = cnxn_es.search(index=idx, body={
        "_source": {
            "excludes": []
        },
        "query": query,
        "size": 200
    })
    print(f"Total hits: {r['hits']['total']['value']}")  # Adjusted for Elasticsearch 7.x

    for hit in r['hits']['hits']:
        source = hit['_source']
        score = hit['_score']  # Accessing the score for each document

        # Printing all required details including the score
        print(f"Symbol: {source.get('symbol', 'N/A')}, Market Cap: {source.get('marketcap', 'N/A')}, \
    CIK: {source.get('cik', 'N/A')}, Description: {source.get('description', 'N/A')}, Score: {score}")

    # Normalize the results into a DataFrame
    df = pd.json_normalize(r['hits']['hits'])

    rows = []

    # Iterate over each hit and collect required details
    for hit in r['hits']['hits']:
        source = hit['_source']
        row = {
            "score": hit['_score'],  # Include the score
            "symbol": source.get('symbol', 'N/A'),
            "marketcap": source.get('marketcap', 'N/A'),
            "cik": source.get('cik', 'N/A'),
            "description": source.get('description', 'N/A'),
            "sector": source.get('sector', 'N/A'),
            "industry": source.get('industry', 'N/A'),
            "country": source.get('country', 'N/A')
        }
        rows.append(row)

    # Create a DataFrame from the collected rows
    df = pd.DataFrame(rows)

    # Save the DataFrame to an Excel file
    # df.to_excel('es_search_results.xlsx', index=False)

    return r, df

In [ ]:
theme_AI = ("AI, cloud computing, robotics, NLP, Artificial Intelligence, machine learning, neural networks, "
            "deep learning")

# ELSER Model search
query = {
                "text_expansion": {
                    "elser_embedding": {
                        "model_id": ".elser_model_2",
                        "model_text": theme_AI,
                    }
               }
}
r, dfr = esquery1(query, es_idx_utestiam)

output_file = 'AI_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    dfr.to_excel(writer, sheet_name='ELSER', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

In [ ]:
theme_Renew = ("renewable energy, clean energy, solar energy, wind energy, hydroelectric power, biomass energy, "
               "geothermal energy, clean energy, sustainable energy, hydroelectric, electric vehicle, biofuel")

# ELSER Model search
query = {
                "text_expansion": {
                    "elser_embedding": {
                        "model_id": ".elser_model_2",
                        "model_text": theme_Renew,
                    }
               }
}
r, dfr = esquery1(query, es_idx_utestiam)

output_file = 'Renewable_SearchResults.xlsx'

# append data to existing file
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    dfr.to_excel(writer, sheet_name='ELSER', index=False)

print(f"New DataFrame has been successfully added to {output_file}")

## Step 2: AI Manual check
We manually reviewed the search results and added a column called 'Flag' to show. We will use the labeled datasets, both of which are located in the folder. The names of the datasets are AI_SearchResults and Renewable_SearchResults.

Please **change** the file path that contains AI_SearchResults.xlsx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Remeber to change the file path ###
file_path = '/content/drive/MyDrive/thematic/AI/AI_SearchResults.xlsx'

df_els = pd.read_excel(file_path, sheet_name='ELSER')
df_exact = pd.read_excel(file_path, sheet_name='Exact Match')
columns_to_keep = ['symbol', 'marketcap', 'cik', 'description', 'sector', 'industry', 'country', 'Flag']

# company whose flag = 1
df_els_filtered = df_els[df_els['Flag'] == 1][columns_to_keep]
df_exact_filtered = df_exact[df_exact['Flag'] == 1][columns_to_keep]

combined_df = pd.concat([df_els_filtered, df_exact_filtered])

unique_combined_df = combined_df.drop_duplicates(subset='symbol')
unique_combined_df.reset_index(drop=True, inplace=True)
print(unique_combined_df.head())


In [ ]:
unique_combined_df.to_excel('unique_combined_df.xlsx', index=False)

## Step 3 for AI: Bart classification


In [ ]:
!pip install transformers
!pip install torch
!pip install sentence-transformers

In [ ]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer, util

from transformers import pipeline
# Choose the model and task
model_name = 'facebook/bart-large-mnli'
classifier = pipeline('zero-shot-classification', model=model_name)

In [ ]:
# load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
candidate_labels = ["GPU", "Semiconductor vendors", "Data center", "AI software provider", "Computing" ,"AI service", "AI consultancy"]
ai_query = "This sentence is about GPU, artificial intelligence, machine learning, computing, neural networks, and deep learning."

def split_into_sentences(text):
    # Regular expression to split text into sentences
    sentence_endings = re.compile(r'(?<!\bInc)(?<!\bLtd)(?<!\bCo)(?<!\bCorp)(?<!\bInc\.)(?<!\bLtd\.)(?<!\bCo\.)(?<!\bCorp\.)(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    sentences = sentence_endings.split(text)
    return sentences

def extract_ai_sentences(sentences):
    # Encode the sentences
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    query_embedding = model.encode(ai_query, convert_to_tensor=True)
    # Compute cosine similarities
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    # Filter sentences based on cosine similarity
    threshold = 0.3
    ai_sentences = [sentences[i] for i in range(len(sentences)) if cos_scores[i] > threshold]
    return ai_sentences

In [ ]:
def classify_sentences(sentences):
    # Classify sentences using the BART model
    results = []
    for sentence in sentences:
        result = classifier(sentence, candidate_labels=candidate_labels,multi_label=True)
        label = result['labels'][0]
        score = result['scores'][0]
        results.append((label, score))
    return results

In [ ]:
new_rows = []
for index, row in unique_combined_df.iterrows():

    sentences = split_into_sentences(row['description'])
    ai_related_sentences = extract_ai_sentences(sentences)


    sentence_classifications = classify_sentences(ai_related_sentences)

    for sentence, classification in zip(ai_related_sentences, sentence_classifications):
        new_row = {
            'symbol': row['symbol'],
            'marketcap': row['marketcap'],
            'cik': row['cik'],
            'description': row['description'],
            'sector': row['sector'],
            'industry': row['industry'],
            'country': row['country'],
            'sentences': sentence,
            'AI_label': classification[0],  #
            'score': classification[1]       #
        }
        new_rows.append(new_row)

In [ ]:
new_data = pd.DataFrame(new_rows)
classified_data = new_data.sort_values(by='symbol').reset_index()

### Assign score and rank

In [ ]:
label_scores = {
    "GPU": 5,
    "Semiconductor vendors": 4,
    "Data center": 4,
    "AI software provider": 3,
    "Computing": 3.5,
    "AI service": 2,
    "AI consultancy": 1
}

def calculate_exposure_score(label, score):
    if score < 0.75:
        return 0
    else:
        return label_scores.get(label, 0)

In [ ]:
classified_data['exposure_score'] = classified_data.apply(lambda row: calculate_exposure_score(row['AI_label'], row['score']), axis=1)

# Sum up exposure scores for each company
exposure_sum = classified_data.groupby('symbol')['exposure_score'].sum().reset_index()

# Merge exposure_sum back into classified_data on 'symbol'
classified_data = classified_data.merge(exposure_sum, on='symbol', how='left', suffixes=(None, '_sum'))

# Rename the new column to exposure_sum
classified_data.rename(columns={'exposure_score_sum': 'exposure_sum'}, inplace=True)

In [ ]:
classified_data.head()

In [ ]:
merged_data = classified_data.groupby(['symbol', 'marketcap', 'cik', 'description', 'sector', 'industry', 'country', 'exposure_sum']).agg({
    'AI_label': lambda x: ', '.join(set(x)),}).reset_index()
f_list = merged_data.copy()

# Sort by exposure_sum in descending order
f_list.sort_values(by='exposure_sum', ascending=False, inplace=True)
f_list.to_excel('/content/drive/MyDrive/thematic/AI/AI_rec_list.xlsx', index=False)

## Step 2: Renewable Manual check
Please **change** the file path that contains Renewable_SearchResults.xlsx

In [ ]:
import pandas as pd
import re
### Remeber to change the file path ###
file_path = '/content/drive/MyDrive/thematic/Renewable/Renewable_SearchResults.xlsx'
df_els = pd.read_excel(file_path, sheet_name='ELSER')
df_exact = pd.read_excel(file_path, sheet_name='Exact Match')
columns_to_keep = ['symbol', 'marketcap', 'cik', 'description', 'sector', 'industry', 'country', 'Flag']

# company whose flag = 1
df_els_filtered = df_els[df_els['Flag'] == 1][columns_to_keep]
df_exact_filtered = df_exact[df_exact['Flag'] == 1][columns_to_keep]

combined_df = pd.concat([df_els_filtered, df_exact_filtered])

data = combined_df.drop_duplicates(subset='symbol')
data.reset_index(drop=True, inplace=True)
print(data.head())


## Step 3 for Renewable Energy: Keyword Search


In [ ]:
# Define theme keywords
theme_keywords = [
    "solar", "wind", "hydrogen fueling", "hydro", "biomass", "biofuels",
    "geothermal", "ethanol", "natural gas", "oil", "nuclear", "coal", "methane"
]

# Function to tag description based on theme keywords
def tag_description(description, keywords):
    sentences = re.split(r'(?<=[.!?]) +', description)
    tags = []
    for sentence in sentences:
        for keyword in keywords:
            if re.search(keyword, sentence, re.IGNORECASE):
                tags.append(keyword)
    return list(set(tags))  # Return unique tags

# Apply the tag_description function to each description
data['description_tag'] = data['description'].apply(lambda x: tag_description(x, theme_keywords))

# Remove rows with empty description_tag and drop 'normalized score' column
data = data[data['description_tag'].map(len) > 0]
# data = data.drop(columns=['Normalized Score'])

# Define scoring rules
def calculate_score(tags):
    score = 0

    if len(tags) == 1 and tags[0] in ["solar", "wind", "hydrogen fueling", "hydro", "biomass", "biofuels", "geothermal", "ethanol"]:
        score += 50
    if "natural gas" in tags:
        score -= 15
    if 1 <= len(tags) <= 3 and "solar" in tags:
        score += 15
    if 4 <= len(tags) <= 6 and "solar" in tags and not any(kw in tags for kw in ["natural gas", "oil", "nuclear", "coal", "methane"]):
        score += 8
    if 2 <= len(tags) <= 3 and "wind" in tags and not any(kw in tags for kw in ["natural gas", "oil"]):
        score += 15
    if len(tags) == 1 and tags[0] in ["natural gas", "oil"]:
        score -= 50
    if "nuclear" in tags:
        score -= 15
    if "coal" in tags:
        score -= 15
    if 1 <= len(tags) <= 3 and "hydro" in tags:
        score += 15
    if "oil" in tags:
        score -= 15
    if "ethanol" in tags:
        score -= 15
    if "biofuels" in tags:
        score += 15
    if "methane" in tags:
        score -= 15

    return score

# Apply scoring rules to each company
data['score'] = data['description_tag'].apply(lambda x: calculate_score(x))

# Filter out companies in the "Building Products & Equipment" industry
data = data[data['industry'] != "Building Products & Equipment"]

# Separate the calculation of keyword counts
data['keyword_count'] = data['description_tag'].apply(lambda x: len(x))

# Sort by score and keyword count
data = data.sort_values(by=['score', 'keyword_count'], ascending=[False, True])

# Drop the temporary 'keyword_count' column
data = data.drop(columns=['keyword_count'])

# Step 1: Remove companies with score less than 0
data = data[data['score'] >= 0]

# Step 2: Adjust ranking for companies with score 0
zero_score_data = data[data['score'] == 0]
non_zero_score_data = data[data['score'] != 0]

# Further sorting for zero score companies
utilities_regulated_electric = zero_score_data[zero_score_data['industry'] == "Utilitiesâ€”Regulated Electric"]
other_industries = zero_score_data[zero_score_data['industry'] != "Utilitiesâ€”Regulated Electric"]

# Concatenate the dataframes
sorted_zero_score_data = pd.concat([other_industries, utilities_regulated_electric])
data = pd.concat([non_zero_score_data, sorted_zero_score_data])

# Adjust ranking for companies with score 65
score_65_data = data[data['score'] == 65]
non_score_65_data = data[data['score'] != 65]

# Further sorting for score 65 companies
solar_industry = score_65_data[score_65_data['industry'].str.contains("solar", case=False, na=False)]
other_industries = score_65_data[~score_65_data['industry'].str.contains("solar", case=False, na=False)]

# Concatenate the dataframes
sorted_score_65_data = pd.concat([solar_industry, other_industries])
data = pd.concat([sorted_score_65_data, non_score_65_data])

# Adjust ranking for companies with score 0 and sector "Utilities"
zero_score_data = data[data['score'] == 0]
non_zero_score_data = data[data['score'] != 0]

# Further sorting for zero score companies
utilities_sector = zero_score_data[zero_score_data['sector'] == "Utilities"]
other_sectors = zero_score_data[zero_score_data['sector'] != "Utilities"]

# Concatenate the dataframes
sorted_zero_score_data = pd.concat([other_sectors, utilities_sector])
data = pd.concat([non_zero_score_data, sorted_zero_score_data])

# Save the resulting dataframe to an Excel file
final_sorted_excel_file_path = '/content/drive/MyDrive/thematic/Renewable/Renew_rec_list.xlsx'
data.to_excel(final_sorted_excel_file_path, index=False)

print(f"The final sorted data has been saved to: {final_sorted_excel_file_path}")